In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from sklearn.utils import class_weight
from tensorflow.keras.models import load_model


In [2]:
# Paths
images_dir = "data/images"
labels_path = os.path.join(images_dir, "labels.csv")

# Load labels
df = pd.read_csv(labels_path)
print(f"Total images found: {len(df)}")

data = []
labels = []

for index, row in df.iterrows():
    img_path = os.path.join(images_dir, row['filename'])
    img = cv2.imread(img_path)
    if img is None:
        print(f"Skipped: {img_path}")
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (128, 128))
    data.append(resized)
    labels.append(0 if row['label'] == 'Happy' else 1)  # 0=Happy, 1=Sad

# Convert to numpy arrays
data = np.array(data, dtype='float32') / 255.0
data = np.expand_dims(data, axis=-1)  # shape: (N, 128, 128, 1)
labels = to_categorical(np.array(labels), num_classes=2)

# Train-test split
trainX, testX, trainY, testY = train_test_split(
    data, labels, test_size=0.2, stratify=labels, random_state=42
)

print(f"✅ Data ready: {trainX.shape[0]} training, {testX.shape[0]} testing")


Total images found: 165
✅ Data ready: 132 training, 33 testing


In [3]:
model = Sequential([
    Input(shape=(128, 128, 1)),
    Conv2D(32, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(2, activation="softmax")
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,482,050 (32.36 MB)

 Trainable params: 8,482,050 (32.36 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(np.argmax(trainY, axis=1)),
    y=np.argmax(trainY, axis=1)
)
class_weights = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    trainX, trainY,
    validation_data=(testX, testY),
    epochs=5,   
    batch_size=32,
    class_weight=class_weights
)


Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.6288 - loss: 0.6822 - val_accuracy: 0.7576 - val_loss: 0.6723
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.6364 - loss: 0.6772 - val_accuracy: 0.6970 - val_loss: 0.6436
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.6667 - loss: 0.6624 - val_accuracy: 0.7273 - val_loss: 0.6015
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.6742 - loss: 0.6425 - val_accuracy: 0.6364 - val_loss: 0.5745
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.6818 - loss: 0.6272 - val_accuracy: 0.6667 - val_loss: 0.5435


In [6]:
model.save("happy_sad_model.h5")

loss, acc = model.evaluate(testX, testY)
print(f"Test Accuracy: {acc*100:.2f}%")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6667 - loss: 0.5435
Test Accuracy: 66.67%


In [7]:
# Example: test a single image
test_img_path = "data/images/test_images/sadyellow.jpg"
img = cv2.imread(test_img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
resized = cv2.resize(gray, (128, 128))
input_img = np.expand_dims(resized, axis=(0, -1)) / 255.0

pred = model.predict(input_img)
result = ["Happy", "Sad"][np.argmax(pred)]
print(f"Prediction: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Prediction: Sad


The above output was correct.